In [139]:
# import the necessary packages
from imutils import face_utils
import dlib
import cv2
import numpy as np
import math

In [2]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
p = "shape_predictor_194_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

#### 特征四：眼球中心到眉毛中心

In [126]:
#假设一个视频中人数恒定且相应人脸可以对应上,最终返回特征为eye2brow
cap = cv2.VideoCapture('C:/Users/balabalr/Documents/acxwigylke.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

eye2browDis = []
for f in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)


    # loop over the face detections
    dis = []
    l_eye_feature=[element for element in range(48,70)]
    l_eye_feature.remove(55)
    l_eye_feature.remove(66)
    r_eye_feature=[element for element in range(26,48)]
    r_eye_feature.remove(33)
    r_eye_feature.remove(44)
    r_brow_feature=[element for element in range(70,92)]
    r_brow_feature.remove(77)
    r_brow_feature.remove(88)
    l_brow_feature=[element for element in range(92,114)]
    l_brow_feature.remove(99)
    l_brow_feature.remove(110)
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        series=[]
        #找出眼睛的中心
        for i in range(20):
            series.append([shape[l_eye_feature[i]],shape[r_eye_feature[i]],shape[l_brow_feature[i]],shape[r_brow_feature[i]]])
        series=np.array(series)
        l_eye_center=[(np.max(series[:,0,0])+np.min(series[:,0,0]))/2,(np.max(series[:,0,1])+np.min(series[:,0,1]))/2]
        r_eye_center=[(np.max(series[:,1,0])+np.min(series[:,1,0]))/2,(np.max(series[:,1,1])+np.min(series[:,1,1]))/2]
        l_brow_center=[(np.max(series[:,2,0])+np.min(series[:,2,0]))/2,(np.max(series[:,2,1])+np.min(series[:,2,1]))/2]
        r_brow_center=[(np.max(series[:,3,0])+np.min(series[:,3,0]))/2,(np.max(series[:,3,1])+np.min(series[:,3,1]))/2]
        l_eye_center=np.array(l_eye_center)
        l_brow_center=np.array(l_brow_center)
        r_eye_center=np.array(r_eye_center)
        r_brow_center=np.array(r_brow_center)
        dis.append((np.sqrt(np.sum((l_eye_center - l_brow_center) ** 2))+np.sqrt(np.sum((r_eye_center - r_brow_center) ** 2)))/2)
        
    eye2browDis.append(dis)
#Dis为二维数组[帧数，脸数]
eye2browDis = np.array(eye2browDis)
eye2brow = 0
for i in range(len(rects)):
    eye2brow += eye2browDis[:,i].std()
#求几张脸该特征的均值
eye2brow = eye2brow/len(rects)
print(eye2brow)

cap.release()


0.8760680332076587


特征五：鼻梁和眉毛夹角


In [154]:
#计算两向量的夹角
def angle_new(v1, v2):
    dx1 = v1[2] - v1[0]
    dy1 = v1[3] - v1[1]
    dx2 = v2[2] - v2[0]
    dy2 = v2[3] - v2[1]
    angle1 = math.atan2(dy1, dx1)
    angle1 = int(angle1 * 180/math.pi)
    # print(angle1)
    angle2 = math.atan2(dy2, dx2)
    angle2 = int(angle2 * 180/math.pi)
    # print(angle2)
    if angle1*angle2 >= 0:
        included_angle = abs(angle1-angle2)
    else:
        included_angle = abs(angle1) + abs(angle2)
        if included_angle > 180:
            included_angle = 360 - included_angle
    return included_angle
'''
AB = [5,-1,1,-3]
CD = [4,1,4.5,4.5]
print(angle_new(AB, CD),type(AB))
'''

126 <class 'list'>


In [167]:
#假设一个视频中人数恒定且相应人脸可以对应上,最终返回特征为angle_nose_brow
cap = cv2.VideoCapture('C:/Users/balabalr/Documents/acxwigylke.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

langle=[]
rangle=[]
for f in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)


    # loop over the face detections
    langle_nose_brow = []
    rangle_nose_brow = []
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        nose_vec=[(shape[139][0]+shape[147][0])/2,(shape[139][1]+shape[147][1])/2,(shape[135][0]+shape[151][0])/2,(shape[135][1]+shape[151][1])/2]
        l_brow_vec=[shape[103][0],shape[103][1],shape[113][0],shape[113][1]]
        r_brow_vec=[shape[81][0],shape[81][1],shape[91][0],shape[91][1]]
        langle_nose_brow.append(angle_new(nose_vec,l_brow_vec))
        rangle_nose_brow.append(angle_new(nose_vec,r_brow_vec))
    langle.append(langle_nose_brow)
    rangle.append(rangle_nose_brow)
#Dis为二维数组[帧数，脸数]
langle = np.array(langle)
langle_nose_brow = 0
rangle = np.array(rangle)
rangle_nose_brow = 0
for i in range(len(rects)):
    langle_nose_brow += langle[:,i].std()
    rangle_nose_brow += rangle[:,i].std()
#求几张脸该特征的均值
langle_nose_brow = langle_nose_brow/len(rects)
rangle_nose_brow = rangle_nose_brow/len(rects)
print(langle_nose_brow,rangle_nose_brow)

cap.release()


[[84]]
[[84], [83]]
[[84], [83], [86]]
[[84], [83], [86], [87]]
[[84], [83], [86], [87], [85]]
[[84], [83], [86], [87], [85], [86]]
[[84], [83], [86], [87], [85], [86], [88]]
[[84], [83], [86], [87], [85], [86], [88], [86]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85], [87]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85], [87], [86]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85], [87], [86], [86]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85], [87], [86], [86], [86]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85], [87], [86], [86], [86], [87]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85], [87], [86], [86], [86], [87], [87]]
[[84], [83], [86], [87], [85], [86], [88], [86], [87], [85], [85]

KeyboardInterrupt: 

In [162]:
print(angle_new([652.0, 697.5, 652.5, 677.5],[710, 644, 670, 644]))

92


特征六：眉毛粗细

In [145]:
#假设一个视频中人数恒定且相应人脸可以对应上,最终返回特征为brow_thickness
cap = cv2.VideoCapture('C:/Users/balabalr/Documents/acxwigylke.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

each_brow_thickness = []
for f in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)


    # loop over the face detections
    thickness = []
    each_thickness=[]
    r_brow_feature=[element for element in range(70,92)]
    r_brow_feature.remove(77)
    r_brow_feature.remove(88)
    l_brow_feature=[element for element in range(92,114)]
    l_brow_feature.remove(99)
    l_brow_feature.remove(110)
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        for i in range(10):
            each_thickness.append(np.linalg.norm(np.concatenate((shape[r_brow_feature[i]],shape[r_brow_feature[-i-1]])).mean(axis=0)))
            each_thickness.append(np.linalg.norm(np.concatenate((shape[l_brow_feature[i]],shape[l_brow_feature[-i-1]])).mean(axis=0)))
        thickness.append(np.mean(each_thickness))
        
    each_brow_thickness.append(thickness)
#Dis为二维数组[帧数，脸数]
each_brow_thickness = np.array(each_brow_thickness)
brow_thickness = 0
for i in range(len(rects)):
    brow_thickness += each_brow_thickness[:,i].std()
#求几张脸该特征的均值
brow_thickness = brow_thickness/len(rects)
print(brow_thickness)

cap.release()

KeyboardInterrupt: 